In [5]:
from utilsgemini import *

# Step 1: Process reference papers
print("Processing reference papers...")
documents = process_reference_papers_txt(txt_folder)

# Step 2: Build vector store
print("Building vector store...")
vector_store = build_vector_store(documents)

# Step 3: Input introduction section
intro_text = """ text with [ref1] indicators... """
    
cleaned_intro, ground_truth = preprocess_introduction(intro_text)

# Step 4: Predict references
print("Predicting references...")
faiss_references_dict = predict_references_with_RAG(vector_store, cleaned_intro)

# Step 5: Assess predictions (if in assessment mode)
print("Assessing predictions...")
detailed_report, overall_score = evaluate_predictions(ground_truth, faiss_references_dict)

# Output results
print("\nDetailed Report:")
for item in detailed_report:
    print(item)
print(f"\noverall_score: {overall_score*100:.2f}%")
